In [ ]:
#import os
#assert os.environ['COLAB_TPU_ADDR'], 'Make sure to select TPU from Edit > Notebook settings > Hardware accelerator'
#VERSION = "nightly"  #@param ["1.5" , "20200325", "nightly"]
#!curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
#!python pytorch-xla-env-setup.py --version $VERSION
#import torch_xla
#import torch_xla.core.xla_model as xm

!pip install omegaconf
!pip install optuna

import random

import matplotlib.pyplot as plt
import numpy as np
from omegaconf import OmegaConf
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
# import wandb
import optuna





class Net(nn.Module):
    def __init__(self,trial):
        super(Net, self).__init__()
        self.lstm = nn.LSTM(1, 42, num_layers=1, batch_first=True, bidirectional=False)
        
        #n_units_FC2 = trial.suggest_int("n_units_FC2", 4, 64)
        #n_units_FC3 = trial.suggest_int("n_units_FC3", 4, 64)
        #n_units_FC4 = trial.suggest_int("n_units_FC4", 4, 64)

        n_units_FC2 = 25
        n_units_FC3 = 25
        n_units_FC4 = 25

        self.fc_layers = nn.Sequential(
            nn.Linear(42, n_units_FC2),
            nn.ReLU(),
            nn.Linear(n_units_FC2, 2),
            nn.ReLU(),
            nn.Linear(2, n_units_FC3),
            nn.ReLU(),
            nn.Linear(n_units_FC3, n_units_FC4),
            nn.ReLU(),
            nn.Linear(n_units_FC4, 1)
        )

    def forward(self, x):
        x, _ = self.lstm(x)
        x = x[:, -1, :] # Get last output only (many-to-one)
        x = self.fc_layers(x)
        return x


def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


def get_dataset(data_length=400, sample_rate=2e-6):
    # Load CSV Files
    # TODO: Use numpy instead of pandas
    
    V = pd.read_csv('drive/My Drive/Colab Notebooks/Data/Dummy_volt.csv', header=None).iloc[:,0:data_length]
    # B = pd.read_csv('D:\Dropbox (Princeton)\MagNet_TestData_dummy\Dummy_flux.csv', header=None)
    # F = pd.read_csv('D:\Dropbox (Princeton)\MagNet_TestData_dummy\Dummy_freq.csv', header=None)
    P = pd.read_csv('drive/My Drive/Colab Notebooks/Data/Dummy_loss.csv', header=None)

    # F = F.apply(np.log10)
    # B = B.apply(np.log10)
    P = P.apply(np.log10)

    I = pd.concat([P], axis=1, ignore_index=True)

    # Reshape data
    in_tensors = torch.from_numpy(V.to_numpy()).view(-1, data_length, 1)
    out_tensors = torch.from_numpy(I.to_numpy()).view(-1, 1)

    # Save data as CSV
    np.save("dataset.lstm.in.npy", in_tensors.numpy())
    np.save("dataset.lstm.out.npy", out_tensors.numpy())

    return torch.utils.data.TensorDataset(in_tensors, out_tensors)


def load_dataset(in_filename="dataset.lstm.in.npy", out_filename="dataset.lstm.out.npy"):
    in_tensors = torch.from_numpy(np.load(in_filename))
    out_tensors = torch.from_numpy(np.load(out_filename))

    return torch.utils.data.TensorDataset(in_tensors, out_tensors)


def objective(trial):
    # Load Configuration
    YAML_CONFIG = OmegaConf.load("drive/My Drive/Colab Notebooks/lstm.yaml")
    CLI_CONFIG = OmegaConf.from_cli()
    DEFAULT_CONFIG = OmegaConf.merge(YAML_CONFIG, CLI_CONFIG)

    # Get hyperparameters from Optuna
    OPTUNA_CONFIG = OmegaConf.create({
        "LR": trial.suggest_loguniform("LR", 1e-5, 1e-2),
    })
    CONFIG = OmegaConf.merge(DEFAULT_CONFIG, OPTUNA_CONFIG)

    # Reproducibility
    random.seed(CONFIG.SEED)
    np.random.seed(CONFIG.SEED)
    torch.manual_seed(CONFIG.SEED)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

    # Setup CPU or GPU
    if CONFIG.USE_GPU and not torch.cuda.is_available():
        raise ValueError("GPU not detected but CONFIG.USE_GPU is set to True.")
    device = torch.device("cuda")
    #device = torch.device("cpu")
    #device = xm.xla_device()

    # Setup dataset and dataloader
    # NOTE(seungjaeryanlee): Load saved dataset for speed
    dataset = get_dataset()
    #dataset = load_dataset()
    train_size = int(0.6 * len(dataset))
    valid_size = int(0.2 * len(dataset))
    test_size = len(dataset) - train_size - valid_size
    train_dataset, valid_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, valid_size, test_size])
    kwargs = {'num_workers': 1, 'pin_memory': True} if CONFIG.USE_GPU else {}
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=CONFIG.BATCH_SIZE, shuffle=True, **kwargs)
    valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=CONFIG.BATCH_SIZE, shuffle=False, **kwargs)
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=CONFIG.BATCH_SIZE, shuffle=False, **kwargs)

    # Setup neural network and optimizer
    net = Net(trial).double().to(device)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(net.parameters(), lr=CONFIG.LR)
    # Log number of parameters
    CONFIG.NUM_PARAMETERS = count_parameters(net)

    # Setup wandb
    # wandb.init(project="MagNet", config=CONFIG)
    # wandb.watch(net)

    # Training
    for epoch_i in range(1, CONFIG.NUM_EPOCH+1):
        # Train for one epoch
        epoch_train_loss = 0
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = net(inputs.to(device))
            loss = criterion(outputs, labels.to(device))
            loss.backward()
            optimizer.step()

            epoch_train_loss += loss.item()

        # Compute Validation Loss
        with torch.no_grad():
            epoch_valid_loss = 0
            for inputs, labels in valid_loader:
                outputs = net(inputs.to(device))
                loss = criterion(outputs, labels.to(device))

                epoch_valid_loss += loss.item()

        print(f"Epoch {epoch_i:2d} "
            f"Train {epoch_train_loss / len(train_dataset):.5f} "
            f"Valid {epoch_valid_loss / len(valid_dataset):.5f}")
        # wandb.log({
        #     "train/loss": epoch_train_loss / len(train_dataset),
        #     "valid/loss": epoch_valid_loss / len(valid_dataset),
        # })

    # Evaluation
    net.eval()
    y_meas = []
    y_pred = []
    with torch.no_grad():
        for inputs, labels in test_loader:
            y_pred.append(net(inputs.to(device)))
            y_meas.append(labels.to(device))

    y_meas = torch.cat(y_meas, dim=0)
    y_pred = torch.cat(y_pred, dim=0)
    # print(f"Test Loss: {F.mse_loss(y_meas, y_pred).item() / len(test_dataset):.8f}")
    # wandb.log({"test/loss": F.mse_loss(y_meas, y_pred).item() / len(test_dataset)})

    # Predicton vs Target Plot
    fig, ax = plt.subplots(1, 1)
    fig.set_size_inches(8, 8)
    ax.scatter(y_meas.cpu().numpy(), y_pred.cpu().numpy(), label="Prediction")
    ax.plot(y_meas.cpu().numpy(), y_meas.cpu().numpy(), 'k--', label="Target")
    ax.grid(True)
    ax.legend()
    # wandb.log({"prediction_vs_target": wandb.Image(fig)})
    
    y_pred = y_pred.cpu().numpy()
    y_pred = 10**y_pred
    y_meas = y_meas.cpu().numpy()
    y_meas = 10**y_meas

    # Relative Error
    Error_re1 = abs(y_pred-y_meas)/abs(y_meas)*100
    Error_re1[np.where(Error_re1>500)] = 500
    # Error_re_max1 = np.max(Error_re1);
    Error_re_avg1 = np.mean(Error_re1);
    # print(f"Relative Error: {Error_re_avg1:.8f}")
    # wandb.log({"Relative Error": Error_re_avg1})
    
    # np.savetxt("pred.csv", y_pred.cpu().numpy())
    # np.savetxt("meas.csv", y_meas.cpu().numpy())
    return Error_re_avg1


if __name__ == "__main__":
    study = optuna.create_study(
        study_name = 'LSTM-FC_linear',
        direction="minimize",
        storage = 'sqlite:///example.db',
        load_if_exists=True)
    study.optimize(objective, n_trials=10)

    pruned_trials = [t for t in study.trials if t.state == optuna.structs.TrialState.PRUNED]
    complete_trials = [t for t in study.trials if t.state == optuna.structs.TrialState.COMPLETE]

    print("Study statistics: ")
    print("  Number of finished trials: ", len(study.trials))
    print("  Number of pruned trials: ", len(pruned_trials))
    print("  Number of complete trials: ", len(complete_trials))

    print("Best trial:")
    trial = study.best_trial

    print("  Value: ", trial.value)

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

    optuna.visualization.plot_optimization_history(study)
    #optuna.visualization.plot_contour(study, params=['n_units_FC2', 'n_units_FC3'])
    #optuna.visualization.plot_contour(study, params=['n_units_FC2', 'n_units_FC4'])
    #optuna.visualization.plot_contour(study, params=['n_units_FC3', 'n_units_FC4'])

[I 2020-07-29 22:15:25,320] Using an existing study with name 'LSTM-FC_linear' instead of creating a new one.


Epoch  1 Train 0.03645 Valid 0.00832
Epoch  2 Train 0.00374 Valid 0.00206
Epoch  3 Train 0.00134 Valid 0.00089


KeyboardInterrupt: ignored